In [37]:
!pip3 install pandas
!pip3 install torch
!pip3 install numpy

In [38]:
import pandas as pd
import torch
from torch.nn.functional import pad
import numpy as np

In [39]:
filename = "preprocessed.csv"
df = pd.read_csv(filename)

Filter

In [40]:
df = df[df.num_tokens > 0]
author_count = df['author'].value_counts()
many_works_author = author_count[author_count >= 10]
df = df[df.author.isin(many_works_author.index.to_numpy())].reset_index()
len(df)

5470

                                    Take the first 512 tokens 

                                    Take the last 512 tokens 

                                    Take the first 128 and last 384 tokens

In [41]:
def get_first_512_tokens(text):
    tokens = text.split()
    if (len(tokens) >= 512):
        return ' '.join(text.split()[:512])
    padding = ['0'] * (512 - len(tokens))
    return ' '.join(tokens + padding)

In [42]:
def get_last_512_tokens(text):
    tokens = text.split()
    if (len(tokens) >= 512):
        return ' '.join(text.split()[-512:])
    padding = ['0'] * (512 - len(tokens))
    return ' '.join(padding + tokens)

In [43]:
def get_head_128_last_384_tokens(text):
    tokens = text.split()
    if (len(tokens) >= 512):
        return ' '.join(text.split()[:128] + text.split()[-384:])
    padding = ['0'] * (512 - len(tokens))
    return ' '.join(tokens + padding)

In [44]:
df['text_head_512'] = df['text'].apply(lambda x: get_first_512_tokens(x))
print(df.iloc[0]['text_head_512'])

transcriber notes page scan source http the diphthong oe is represented by oe see footnote explaining correction of printing error books by hermann sudermann published by charles scribner sons the joy of living lebe das a play in five acts translated from the german by edith wharton roses four plays translated from the german by grace frank morituri three plays translated from the german by archibald alexander morituri morituri three plays teja fritzchen the eternal masculine by hermann sudermann translated from the german by archibald alexander charles scribner sons new york copyright by charles scribner sons published september contents teja fritzchen the eternal masculine i teja a drama in one act persons teja king of the goths balthilda queen amalaberga her mother agila bishop euric theodemir lords in the former kingdom of the goths athanaric ildibad spearbearer of the king haribalt a warrior two camp watchers teja scene represents the king tent the curtains are open in the backgro

In [45]:
df['text_last_512'] = df['text'].apply(lambda x: get_last_512_tokens(x))
print(df.iloc[0]['text_last_512'])

one can walk about it and find something to sketch by the way the marshal how would it be if you went with me the painter where the marshal to the camp the painter yes and what is there the marshal plenty for you you will find gay fare and pastimes and diversions as much as you want the painter and are there fights too the marshal indeed there are the painter and will there be a bold reconnoissance by night the marshal often the painter capital i will ride with you in my mind eye i see already golden moonrise and silver vapour on the dark alder bush are there also songs and notes of the mandolin the marshal plenty of them the painter hurrah there is music too the marshal and in the by night at the many a tale of human destiny will be unfolded to you the painter a world of pictures and love adventures the marshal if you choose to call them adventures the painter agreed sir and an excess of happiness will flow out of my soul like a yet it seems i am forgetting the greatest happiness i sh

In [46]:
df['text_head_last_512'] = df['text'].apply(lambda x: get_head_128_last_384_tokens(x))
print(df.iloc[0]['text_head_last_512'])

transcriber notes page scan source http the diphthong oe is represented by oe see footnote explaining correction of printing error books by hermann sudermann published by charles scribner sons the joy of living lebe das a play in five acts translated from the german by edith wharton roses four plays translated from the german by grace frank morituri three plays translated from the german by archibald alexander morituri morituri three plays teja fritzchen the eternal masculine by hermann sudermann translated from the german by archibald alexander charles scribner sons new york copyright by charles scribner sons published september contents teja fritzchen the eternal masculine i teja a drama in one act persons teja king of the goths balthilda queen amalaberga her mother agila bishop euric theodemir lords in music too the marshal and in the by night at the many a tale of human destiny will be unfolded to you the painter a world of pictures and love adventures the marshal if you choose to 

In [47]:
df.columns

Index(['index', 'id', 'title', 'author', 'authoryearofbirth',
       'authoryearofdeath', 'idx', 'num_tokens', 'num_unique', 'text',
       'text_head_512', 'text_last_512', 'text_head_last_512'],
      dtype='object')

                                                            One hot

In [48]:
def onehot_to_vertor(x):
    values = list(x.values)
    return values

In [49]:
def save_author_order(x):
    x.to_csv('order.csv', index=False)
    

In [50]:
one_hot = pd.get_dummies(df['author'])
save_author_order(one_hot)
# convert true/false to 1/0
one_hot = one_hot.astype(int)
df['label'] = one_hot.apply(onehot_to_vertor, axis=1)
df['label']




0       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
4       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
                              ...                        
5465    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
5466    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
5467    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
5468    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
5469    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: label, Length: 5470, dtype: object

In [51]:
head_df = df[['label', 'text_head_512']]
head_df = head_df.rename(columns={'text_head_512': 'text'})
tail_df = df[['label', 'text_last_512']]
tail_df = tail_df.rename(columns={'text_last_512': 'text'})
head_tail_df = df[['label', 'text_head_last_512']]
head_tail_df = head_tail_df.rename(columns={'text_head_last_512': 'text'})

head_df.to_csv('head.csv', index=False)
tail_df.to_csv('tail.csv', index=False)
head_tail_df.to_csv('head_tail.csv', index=False)

                                                Sampling

In [52]:
!pip3 install scikit-learn

In [53]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import LabelBinarizer

In [54]:
def get_y(x):
    preds = np.argmax(x, axis=0)
    return preds

In [55]:
def sampling(df):
    raw_train_data, test_data = train_test_split(df, test_size=0.2, shuffle=True)
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    X = list(raw_train_data['text'])
    y = raw_train_data['label'].apply(lambda x : get_y(x))
    y = np.array(y)
    train_ds = pd.DataFrame(columns=['text', 'label']).reset_index(drop=True)
    val_ds = pd.DataFrame(columns=['text', 'label']).reset_index(drop=True)
    df = raw_train_data
    for train_idxs, val_idxs in skf.split(X, y):
        for i in train_idxs:
            train_ds = train_ds.append(df.iloc[i], ignore_index=True)
            #train_ds = pd.concat([train_ds, df.iloc[i]], ignore_index=True)
        for i in val_idxs:
            val_ds = val_ds.append(df.iloc[i], ignore_index=True)
            #val_ds = pd.concat([val_ds, df.iloc[i]], ignore_index=True)
    return train_ds, val_ds, test_data

In [56]:
train_ds, val_ds, test_data = sampling(head_df)

/var/folders/h4/wblr9zyj0633z0hcpl2g881w0000gn/T/ipykernel_63033/2192322026.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_ds = train_ds.append(df.iloc[i], ignore_index=True)
/var/folders/h4/wblr9zyj0633z0hcpl2g881w0000gn/T/ipykernel_63033/2192322026.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  val_ds = val_ds.append(df.iloc[i], ignore_index=True)


In [57]:
print(len(test_data), len(train_ds), len(val_ds))
test_data

1094 17504 4376


,label,text
2050,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",lancashire illustration emigrants at liverpool...
1221,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",produced from images available at the internet...
5255,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",the poems of madison cawein volume ii new worl...
572,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",produced from scanned images of public domain ...
1151,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",generously made available by internet librarie...
...,...,...
4709,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",of william shakespeare illustration the works ...
3091,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",produced from images generously made available...
1361,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",the miracle songs of jesus by wilson macdonald...
1684,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",transcriber note text enclosed by underscores ...
